In [12]:
import os
from abc import ABC, abstractmethod
import numpy as np
import random
import time
import sys
import math
class SimpleTokenizer:
    '''
    Class to get the resources from the resource file
    Variables
    text -> which will contain the string from which we wnat to extract data
    separator -> these are the separator which are use in resource file to separate the data
    '''
    start = -1
    end = -1

    def __init__(self, text, separators=[' ', '\t', '\n']):
        self.text = text.replace('\r\n', '')
        self.separators = separators
        self.start = -1
        self.end = -1

    def hasToken(self):
        self.start = self.end + 1
        while (self.start < len(self.text) and self.isSeparator(self.text[self.start])):
            self.start += 1
        if (self.start < len(self.text)):
            self.end = self.start - 1
            return True
        return False

    def nextInt(self):
        while (self.start < len(self.text)):
            try:
                token = self.nextToken().strip()
                return int(token)
            except:
                pass
        raise Exception("NoSuchElement")

    def nextToken(self):
        self.skipToken()
        return self.text[self.start:self.end]

    def skipToken(self):
        self.start = self.end + 1
        while (self.start < len(self.text) and self.isSeparator(self.text[self.start])):
            self.start += 1
        self.end = self.start + 1
        while (self.end < len(self.text) and not self.isSeparator(self.text[self.end])):
            self.end += 1

    def isSeparator(self, character):
        if character in self.separators:
            return True
        return False
class Problem:
    '''
    Array Representation of the problem
    Variables:
    nJobs -> Number of Jobs
    nMachines -> Number of Machines
    processTimes -> Matrix with the process time of a job in a machine , processTimes[machine][job]
    setupTimes -> Matrix with the setup times of the next job considering the current job on same machine
    '''

    def __init__(self, instancePath):
        try:
            print(os.getcwd()+'/'+instancePath)
            with open(os.getcwd()+'/'+instancePath, 'r', newline='') as file:
                self.token = SimpleTokenizer(file.readline())
                self.nJobs = self.token.nextInt()
                self.nMachines = self.token.nextInt()
                # print('Jobs>>'+str(self.nJobs))
                
                self.processTimes = [[False for i in range(self.nJobs)] for j in range(self.nMachines)]
                
                self.setupTimes = [[[False for i in range(self.nJobs)] for j in range(self.nJobs)]for k in range(self.nMachines)]
                file.readline()
                for job in range(0, self.nJobs):
                    self.token = SimpleTokenizer(file.readline())
                    for machine in range(0, self.nMachines):
                        machineid = self.token.nextInt()
                        assert machine == machineid, "machine doesnot match ID in file"
                        self.processTimes[machine][job] = self.token.nextInt()
                file.readline()
                for machine in range(0, self.nMachines):
                    file.readline()
                    for job in range(0, self.nJobs):
                        self.token = SimpleTokenizer(file.readline())
                        for nextJob in range(0, self.nJobs):
                            self.setupTimes[machine][job][nextJob] = self.token.nextInt(
                            )
                            assert job != nextJob or self.setupTimes[machine][job][
                                nextJob] == 0, "setup between equal jobs must be zero"
        except Exception as e:
            print(e)

    def __str__(self):
        print('Array representation of Resource file')
        print('Number of Jobs => ', self.nJobs)
        print('Number of Machines => ', self.nMachines)
        print('Process Time of Jobs => ', self.processTimes)
        print('Setup Time of Jobs=>',self.setupTimes)
        return ""

In [13]:


class Particle:
    def __init__(self, schedule):
        self.schedule = schedule
        self.fitness = calculate_fitness(schedule, processing_times, setup_times)
#         self.velocity = [0 for _ in range(len(schedule))]
        self.velocity = np.zeros((len(schedule), 2), dtype=np.int32)
        self.best_schedule = self.schedule

    def update_velocity(self, global_best):
        w, c1, c2 = 0.8, 2, 2  # PSO parameters
        for i in range(len(self.schedule)):
            r1, r2 = random.random(), random.random()
#             print(self.best_schedule)
            cognitive = c1 * r1 * (self.best_schedule[i] - self.schedule[i])
            social = c2 * r2 * (global_best[i] - self.schedule[i])
            self.velocity[i] = w * self.velocity[i] + cognitive + social

    def update_position(self):
#         print('self.velocity',self.velocity)
        for i in range(len(self.schedule)):
            self.schedule[i] += (self.velocity[i])
            if not (self.schedule[i][0]>=0 and self.schedule[i][0]<num_machines):
                self.schedule[i][0] = random.randint(0, num_machines-1)
        self.fitness = calculate_fitness(self.schedule, processing_times, setup_times)
    def subtract_two_tuple(x,y):
        return (x[0]-y[0],x[1]-y[1])


In [14]:
def calculate_fitness(schedule, processing_times, setup_times):
    # initialize the completion times of each job on each machine
#     completion_times = [[0] * len(processing_times[0]) for _ in range(len(processing_times))]
    completion_times = [[0] * problem.nMachines for _ in range(len(schedule))]
    machine_done = []
    machines = [[0] for _ in range(problem.nMachines)]
#     print('sechd',schedule)
    for k in schedule:
        machines[k[0]] += [k[1]]
    for i in range(len(machines)):
        machines[i].pop(0)
#     print('completion_times',completion_times)
    # loop through each job in the schedule
    for i in range(len(schedule)):
        # get the machine and processing time for the job
        machine = schedule[i][0]
#         processing_time = processing_times[i][machine]
        processing_time = processing_times[machine][i]
        # calculate the start time of the job (maximum of completion time and setup time)
#         start_time = max(completion_times[i][machine], setup_times[i][machine])
        if machine not in machine_done:
            start_time = completion_times[i][machine] + 0
            machine_done.append(machine)
#             start_time = max(completion_times[i][machine], setup_times[machine][i][i])
        else:
            l = machines[machine].index(schedule[i][1])
            h = machines[machine][l-1]
            start_time = completion_times[i][machine] + setup_times[machine][h][i]
#             start_time = max(completion_times[i][machine],setup_times[machine][i-1][i])
        # update the completion time of the job on the machine
        completion_times[i][machine] = start_time + processing_time
        # update the completion times of the remaining jobs on the machine
        for j in range(i+1, len(schedule)):
            if schedule[j][0] == machine:
                completion_times[j][machine] = completion_times[i][machine]
    # calculate the makespan (maximum completion time of all jobs on all machines)
#     print('completion_times',completion_times)
    makespan = max(max(completion_times[i]) for i in range(len(completion_times)))
    # return the fitness (1/makespan, since we want to minimize makespan)
    return 1/makespan

# function to generate a random schedule
def generate_schedule(num_jobs, num_machines):
    schedule = np.zeros((num_jobs, 2), dtype=np.int32)
#     schedule = []
    
    for i in range(num_jobs):
        while True:
            machine = random.randint(0, num_machines-1)
#             cond = False
            for k in schedule:
                if k[0] == machine and k[1] == i:
                    continue
            break
#         schedule.append([machine,i])
        schedule[i][0]= machine
        schedule[i][1] = i
#     print('Schedule',schedule)
    return schedule


In [15]:

# def calculate_fitness(schedule, processing_times, setup_times):
#     # function to calculate the fitness of a schedule
#     # (same as given code)

def generate_random_particle(num_jobs, num_machines):
    schedule = generate_schedule(num_jobs, num_machines)
    particle = Particle(schedule)
    return particle

def run_pso_algorithm(num_jobs, num_machines, processing_times, setup_times, population_size=100, num_generations=500):
    # initialize the population
    population = [generate_random_particle(num_jobs, num_machines) for _ in range(population_size)]
    # set the global best particle
    global_best = population[0].schedule
    for particle in population:
        if particle.fitness > calculate_fitness(global_best, processing_times, setup_times):
            global_best = particle.schedule
    # loop through each generation
    for generation in range(num_generations):
        # update the velocity and position of each particle
        for particle in population:
            particle.update_velocity(global_best)
            particle.update_position()
            # update the particle's best position
            if particle.fitness > calculate_fitness(particle.best_schedule, processing_times, setup_times):
                particle.best_schedule = particle.schedule
            # update the global best position
            if particle.fitness > calculate_fitness(global_best, processing_times, setup_times):
                global_best = particle.schedule
    return global_best


In [16]:
inFile = "resource.txt"
problem = Problem(inFile)

E:\COdes\Jupyter notebook\Course\Genetic Algorithm/resource.txt


In [17]:

num_jobs = problem.nJobs
num_machines = problem.nMachines
processing_times = problem.processTimes
setup_times = problem.setupTimes
# population_size = int(input("Enter the population size: "))
# no_of_generation = int(input("Enter the number of Generations: "))

In [18]:
%%time
random.seed(0)
result = run_pso_algorithm(num_jobs,num_machines,processing_times,setup_times)
print("Result Time",1/calculate_fitness(result,processing_times,setup_times))
print('Schedule0',result)

Result Time 134.0
Schedule0 [[ 3  0]
 [ 2  1]
 [ 2  2]
 [ 3  3]
 [ 1  4]
 [ 3  5]
 [ 4  6]
 [ 1  7]
 [ 0  8]
 [ 2  9]
 [ 4 10]
 [ 0 11]]
Wall time: 17.2 s
